In [12]:
N = ord("E") % 5 + 1
N

5

In [13]:
import pandas as pd

lab6file = pd.read_excel('lab6.xlsx')

In [14]:
lab6file.head(5)

,N,file name,image size,glasses color,line width
0,1,emma-watson2.jpg,300x300,red,2
1,2,emma-watson.jpg,400x400,blue,3
2,3,draco.jpg,700x700,cian,4
3,4,longbottom.jpg,600x600,magenta,5
4,5,ron_wesley.jpg,500x500,yellow,6


In [21]:
import cv2
from PIL import Image, ImageDraw

face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

def add_glasses_to_image(input_image_path, output_image_path, glasses_color, line_thickness, image_size=None):
    original_image = cv2.imread(input_image_path)
    grayscale_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
    
    detected_faces = face_detector.detectMultiScale(grayscale_image, scaleFactor=1.3, minNeighbors=4)

    pil_image = Image.open(input_image_path)
    drawer = ImageDraw.Draw(pil_image)

    for (x, y, width, height) in detected_faces:
        face_region = grayscale_image[y:y + height, x:x + width]

        detected_eyes = eye_detector.detectMultiScale(face_region, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        if len(detected_eyes) < 2:
            continue

        detected_eyes = sorted(detected_eyes, key=lambda eye: eye[0])[:2]
        
        if abs(detected_eyes[0][1] - detected_eyes[1][1]) > 20:
            continue

        left_eye_center = (x + detected_eyes[0][0] + detected_eyes[0][2] // 2, y + detected_eyes[0][1] + detected_eyes[0][3] // 2)
        right_eye_center = (x + detected_eyes[1][0] + detected_eyes[1][2] // 2, y + detected_eyes[1][1] + detected_eyes[1][3] // 2)

        glasses_radius = (right_eye_center[0] - left_eye_center[0]) // 3

        drawer.ellipse((left_eye_center[0] - glasses_radius, left_eye_center[1] - glasses_radius,
                        left_eye_center[0] + glasses_radius, left_eye_center[1] + glasses_radius),
                       outline=glasses_color, width=line_thickness)
        drawer.ellipse((right_eye_center[0] - glasses_radius, right_eye_center[1] - glasses_radius,
                        right_eye_center[0] + glasses_radius, right_eye_center[1] + glasses_radius),
                       outline=glasses_color, width=line_thickness)
        
        drawer.line((left_eye_center[0] + glasses_radius, left_eye_center[1],
                     right_eye_center[0] - glasses_radius, right_eye_center[1]),
                    fill=glasses_color, width=line_thickness)

        drawer.line((left_eye_center[0] - glasses_radius, left_eye_center[1],
                     left_eye_center[0] - glasses_radius - 50, left_eye_center[1]),
                    fill=glasses_color, width=line_thickness)
        drawer.line((right_eye_center[0] + glasses_radius, right_eye_center[1],
                     right_eye_center[0] + glasses_radius + 50, right_eye_center[1]),
                    fill=glasses_color, width=line_thickness)

    if image_size:
        pil_image = pil_image.resize(image_size, Image.Resampling.LANCZOS)
    
    pil_image.save(output_image_path)

In [22]:
add_glasses_to_image(
    "Images/ron_wesley.jpg", 
    "final.jpg", 
    glasses_color="yellow", 
    line_thickness=6, 
    image_size=(500, 500)
)